# Cluster AI 2019 - Grupo 7

## Trabajo práctico integrador -  SUACI

**Etapa 02: Clustering** 

In [1]:
# importamos librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import warnings
warnings.filterwarnings('ignore')

In [2]:
# importamos librerias de clustering
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage 
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn import preprocessing

In [3]:
# importamos el dataset del año 2019
base2019 = pd.read_csv('/Users/fer/Downloads/DATASCIENCE/CLUSTER AI/GRUPO TP/SUACI/sistema-unico-de-atencion-ciudadana-2019.csv', delimiter=',', header = 0, index_col = "contacto")

Para el ejercicio de clusterización nos proponemos trabajar en la identificación de zonas en la Ciudad Autónoma de Buenos Aires con problemáticas similares, definidas las mismas a partir de los contactos realizados al SUACI.
Utilizamos a este efecto la variable "concepto" que aporta la categorización más descriptiva del tipo de contacto, siendo las variables "subcategoría" y "categoría" agrupaciones de esta.
A efectos del problema propuesto no nos interesa el canal a través del cual se realizó el contacto ni el género del denunciante, así como la fecha en que se registró el mismo, en tanto vamos a tomar el horizonte temporal de todos los contactos realizados durante el año calendario 2019.
Definimos entonces un nuevo dataset con las variables estrictamente importantes para nuestro ejercicio, comprendidas por:
- las variables dummies obtenidas a partir de la variable categórica "concepto"
- el par latitud-longitud, para que el algoritmo le asigne también un peso a la proximidad geográfica de los contactos a la hora de clusterizar

In [4]:
# generamos dummies para la categoría "concepto" y los almacenamos en un DF
concepto_dummies = pd.get_dummies(base2019.concepto)
concepto_dummies.head(5)

,ABONOS DE DESCUENTO POR VOLUMEN - SUBTE,ACCESO A LA INFORMACIÓN PÚBLICA,ACCESOS CERRADOS - SUBTE,ACTIVIDAD DE BAILE EFECTUADA SIN PERMISO,ALIMENTOS EN MALAS CONDICIONES,ALMACENAMIENTO DE MATERIALES EN VÍA PÚBLICA,ALTAS - BAJAS TEMPERATURAS - SUBTE,ALTURA ANTIRREGLAMENTARIA EN OBRAS Y CONSTRUCCIONES,ANOMALÍAS EN PUESTO DE DIARIOS,ASCENSORES FUERA DE SERVICIO DURANTE TRÁMITE,...,TERMINALES DE AUTOCONSULTA -SUBTE,TRASCENDENCIA DE CALOR DESDE INSTALACIONES FIJAS COMERCIALES / INDUSTRIALES HACIA INMUEBLES LINDEROS,VACIADO DE CAMPANA VERDE,VEHÍCULO MAL ESTACIONADO,VEHÍCULO OBSTRUYENDO GARAJE,VENTA / EXHIBICIÓN NO PERMITIDA DE BEBIDAS ALCOHÓLICA,VENTA AMBULANTE - SUBTE,VEREDA SUCIA POR MASCOTAS,WIFI - SUBTE,ÁRBOL CON ENFERMEDADES/PLAGAS
contacto,,,,,,,,,,,,,,,,,,,,,
00000001/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000002/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000003/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00000005/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000006/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
# separo las variables latitud y longitud
base2019_clean = base2019.drop(['periodo', 'categoria', 'subcategoria', 'concepto', 'tipo_prestacion', 'fecha_ingreso', 'hora_ingreso', 'domicilio_cgpc', 'domicilio_barrio', 'domicilio_calle', 'domicilio_altura', 'domicilio_esquina_proxima', 'canal', 'genero', 'estado_del_contacto', 'fecha_cierre_contacto'], axis = 1)
base2019_clean.head(5)

,lat,long
contacto,,
00000001/19,-34.634060,-58.466561
00000002/19,-34.628162,-58.443378
00000003/19,-34.623324,-58.469810
00000005/19,-34.618464,-58.445304
00000006/19,-34.634200,-58.528355


In [6]:
# concatenamos ambos dataframes por columnas
base2019_dummies = pd.concat([base2019_clean, concepto_dummies], axis = 1)
base2019_dummies.head(5)

,lat,long,ABONOS DE DESCUENTO POR VOLUMEN - SUBTE,ACCESO A LA INFORMACIÓN PÚBLICA,ACCESOS CERRADOS - SUBTE,ACTIVIDAD DE BAILE EFECTUADA SIN PERMISO,ALIMENTOS EN MALAS CONDICIONES,ALMACENAMIENTO DE MATERIALES EN VÍA PÚBLICA,ALTAS - BAJAS TEMPERATURAS - SUBTE,ALTURA ANTIRREGLAMENTARIA EN OBRAS Y CONSTRUCCIONES,...,TERMINALES DE AUTOCONSULTA -SUBTE,TRASCENDENCIA DE CALOR DESDE INSTALACIONES FIJAS COMERCIALES / INDUSTRIALES HACIA INMUEBLES LINDEROS,VACIADO DE CAMPANA VERDE,VEHÍCULO MAL ESTACIONADO,VEHÍCULO OBSTRUYENDO GARAJE,VENTA / EXHIBICIÓN NO PERMITIDA DE BEBIDAS ALCOHÓLICA,VENTA AMBULANTE - SUBTE,VEREDA SUCIA POR MASCOTAS,WIFI - SUBTE,ÁRBOL CON ENFERMEDADES/PLAGAS
contacto,,,,,,,,,,,,,,,,,,,,,
00000001/19,-34.634060,-58.466561,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000002/19,-34.628162,-58.443378,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000003/19,-34.623324,-58.469810,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00000005/19,-34.618464,-58.445304,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000006/19,-34.634200,-58.528355,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [7]:
base2019_dummies.shape

(626101, 215)

¿Corresponde estandarizar los datos para la clusterización? <br>
Nuestro dataset se compone de una matriz sparse con las variables dummies, más el par de coordenadas latitud y longitud. <br>
El riesgo de no estandarizar es que la diferencia en los órdenes de magnitud de las variables distorsionen las métricas de proximidad del algoritmo de clustering.<br>
Nosotros tenemos por un lado las variables dummies (binarias, con valores 0 y 1), y el par de coordenadas posicionales. <br>
Las coordenadas latitud y longitud presentan problemas de continuidad (por ejemplo, en el caso del antemeridiano de Greenwich, donde la longitud toma valores de +180 y -180 a ambos lados, representando un salto gigante en la distancia para dos puntos próximos entre sí). Restringiendo el dominio a la Ciudad de Buenos Aires, no tenemos este problema, y por otro lado, al expresarse los valores en decimales, la diferencia máxima de los valores en este dominio es inferior a la unidad. Por tanto, no hay diferencias en el orden de magnitud de la diferencia entre las variables. <br>
Evaluamos a continuación los scores obtenidos en la clusterización con los datos originales y normalizados, con apertura sobre el tipo de variables involucradas.

In [8]:
# defino un subconjunto con las coordenadas de latitud y longitud únicamente
base_coord = base2019_dummies.iloc[:,0:2]
base_coord.head(5)

,lat,long
contacto,,
00000001/19,-34.634060,-58.466561
00000002/19,-34.628162,-58.443378
00000003/19,-34.623324,-58.469810
00000005/19,-34.618464,-58.445304
00000006/19,-34.634200,-58.528355


In [9]:
# replico para las variables dummies generadas a partir del campo "concepto"
base_concepto = base2019_dummies.iloc[:,2:]
base_concepto.head(5)

,ABONOS DE DESCUENTO POR VOLUMEN - SUBTE,ACCESO A LA INFORMACIÓN PÚBLICA,ACCESOS CERRADOS - SUBTE,ACTIVIDAD DE BAILE EFECTUADA SIN PERMISO,ALIMENTOS EN MALAS CONDICIONES,ALMACENAMIENTO DE MATERIALES EN VÍA PÚBLICA,ALTAS - BAJAS TEMPERATURAS - SUBTE,ALTURA ANTIRREGLAMENTARIA EN OBRAS Y CONSTRUCCIONES,ANOMALÍAS EN PUESTO DE DIARIOS,ASCENSORES FUERA DE SERVICIO DURANTE TRÁMITE,...,TERMINALES DE AUTOCONSULTA -SUBTE,TRASCENDENCIA DE CALOR DESDE INSTALACIONES FIJAS COMERCIALES / INDUSTRIALES HACIA INMUEBLES LINDEROS,VACIADO DE CAMPANA VERDE,VEHÍCULO MAL ESTACIONADO,VEHÍCULO OBSTRUYENDO GARAJE,VENTA / EXHIBICIÓN NO PERMITIDA DE BEBIDAS ALCOHÓLICA,VENTA AMBULANTE - SUBTE,VEREDA SUCIA POR MASCOTAS,WIFI - SUBTE,ÁRBOL CON ENFERMEDADES/PLAGAS
contacto,,,,,,,,,,,,,,,,,,,,,
00000001/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000002/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000003/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00000005/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
00000006/19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [12]:
# corremos a continuación modelos de clusterizacion sobre los subconjuntos
# se determina de manera arbitraria k=10 para el nro. de clusters
base10 = KMeans(n_clusters=10, random_state=14).fit(base2019_dummies)
base10 = base10.labels_
coord10 = KMeans(n_clusters=10, random_state=14).fit(base_coord)
coord10 = coord10.labels_
concepto10 = KMeans(n_clusters=10, random_state=14).fit(base_concepto)
concepto10 = concepto10.labels_

In [13]:
# calculamos el SS para los modelos obtenidos
score_base10 = silhouette_score(base2019_dummies, base10, sample_size=156525)
score_coord10 = silhouette_score(base_coord, coord10, sample_size=156525)
score_concepto10 = silhouette_score(base_concepto, concepto10, sample_size=156525)

In [16]:
# comparo los resultados obtenidos
print(score_base10)
print(score_coord10)
print(score_concepto10)

0.6623430558672089
0.36609468268997686
0.6846313130703465


In [17]:
# ¿qué pasa si normalizamos las variables?
scaler = preprocessing.StandardScaler().fit(base2019_dummies)
base_scal = scaler.transform(base2019_dummies)

In [18]:
print(base_scal.mean(axis=0))
print(base_scal.std(axis=0))

[-8.17018206e-14  3.57657941e-14  1.04975400e-18 -5.11145084e-17
 -1.25970480e-18  7.56957751e-18 -1.05315861e-17 -4.99342444e-19
  5.44737212e-18 -1.80217228e-17  1.56611948e-18  2.99038032e-18
 -3.13791331e-18  1.75904725e-18  1.18139883e-17 -4.02878563e-19
 -6.07155018e-18 -6.44605701e-18  1.41404701e-17  1.30396470e-17
  8.23915033e-18  2.74070910e-18  1.95197501e-18 -6.94539945e-18
 -7.35395236e-18 -3.73371964e-18 -5.76513549e-18  2.90526513e-18
  2.58750176e-18  4.15362124e-18  5.40197735e-18 -1.90658024e-17
 -8.37533463e-18 -1.02138227e-19 -2.48252636e-19  2.81447560e-18
 -4.76645060e-18  1.95197501e-18  7.94975869e-18  4.66658212e-17
  2.95973885e-17 -3.64066037e-17  2.85987036e-18 -8.56826240e-18
  1.43901414e-17 -3.22302850e-18  1.12352050e-18  3.86990394e-18
  2.73503475e-17  8.72430691e-19  5.07286529e-17  1.22565873e-18
 -4.47705896e-18  7.26316283e-19  6.76382038e-18 -3.13223897e-18
  2.27966849e-18  5.08421398e-18  3.44432800e-18 -4.93441125e-17
  1.90658024e-18  1.37177

In [19]:
# ejercicio, calculo los valores de media y sdv para la base original
print(base2019_dummies.mean(axis=0))
print(base2019_dummies.std(axis=0))

lat                                                                                                                                                         -34.606402
long                                                                                                                                                        -58.451981
ABONOS DE DESCUENTO POR VOLUMEN - SUBTE                                                                                                                       0.000003
ACCESO A LA INFORMACIÓN PÚBLICA                                                                                                                               0.003202
ACCESOS CERRADOS - SUBTE                                                                                                                                      0.000152
ACTIVIDAD DE BAILE EFECTUADA SIN PERMISO                                                                                                                      0.00035

lat                                                                                                                                                          0.031550
long                                                                                                                                                         0.040697
ABONOS DE DESCUENTO POR VOLUMEN - SUBTE                                                                                                                      0.001787
ACCESO A LA INFORMACIÓN PÚBLICA                                                                                                                              0.056499
ACCESOS CERRADOS - SUBTE                                                                                                                                     0.012317
ACTIVIDAD DE BAILE EFECTUADA SIN PERMISO                                                                                                                     0.018911
ALIM

Para las variables dummies, al ser una matriz sparse, tanto la media como el desvío estándar son cercanos a 0.
En el caso del par de coordenadas, la media está dentro del rango de latitudes/longitud, y el desvío estándar es cercano a 0.

In [20]:
# defino los subconjuntos sobre el dataset normalizado
coord_scal = base_scal[:,0:2]
coord_scal.shape

(626101, 2)

In [23]:
concepto_scal = base_scal[:,2:]
concepto_scal.shape

(626101, 213)

In [24]:
# corremos a continuación modelos de clusterizacion con k=10 sobre los subconjuntos
base10_scal = KMeans(n_clusters=10, random_state=14).fit(base_scal)
base10_scal = base10_scal.labels_
coord10_scal = KMeans(n_clusters=10, random_state=14).fit(coord_scal)
coord10_scal = coord10_scal.labels_
concepto10_scal = KMeans(n_clusters=10, random_state=14).fit(concepto_scal)
concepto10_scal = concepto10_scal.labels_

In [25]:
# calculamos el SS para los nuevos modelos obtenidos
score_base10_scal = silhouette_score(base_scal, base10_scal, sample_size=156525)
score_coord10_scal = silhouette_score(coord_scal, coord10_scal, sample_size=156525)
score_concepto10_scal = silhouette_score(concepto_scal, concepto10_scal, sample_size=156525)

In [26]:
# comparo los resultados obtenidos
print(score_base10_scal)
print(score_coord10_scal)
print(score_concepto10_scal)

-0.09664627108947425
0.36546332188967073
-0.14950984920859503


Algunos comentarios sobre nuestro experimento:
- el Score obtenido para el dataset con el par de coordenadas no parece ser afectado por la normalización
- sí se aprecia una diferencia notoria para el subconjunto de variables dummies
- el Score sobre la base conjunta, al componerse de estos dos subconjuntos, se ve afectado en consecuencia
- vamos a avanzar con la clusterización sobre la base sin normalizar por ser la que presenta mejores resultados

In [27]:
# lista donde vamos a guardar las etiquetas generadas por el modelo de clustering
labels = []

In [28]:
# generamos sucesivos modelos de clusterización, en busca del hiperparámetro k que arroje el nro. óptimo de clusters
# iteramos el número de clusters en el rango 2-20
start_time = timeit.default_timer()
for num in range (2, 21):
  kmeans = KMeans(n_clusters=num, random_state=14)
  kmeans.fit(base2019_dummies)
  labels.append(kmeans.labels_)
elapsed = timeit.default_timer()-start_time
print(elapsed)

1027.9879098370002


In [29]:
# observamos las etiquetas generadas
labels

[array([1, 1, 0, ..., 0, 0, 1], dtype=int32),
 array([1, 1, 0, ..., 0, 0, 1], dtype=int32),
 array([2, 2, 1, ..., 1, 1, 2], dtype=int32),
 array([2, 2, 3, ..., 3, 3, 2], dtype=int32),
 array([0, 0, 4, ..., 4, 4, 0], dtype=int32),
 array([1, 1, 5, ..., 5, 5, 1], dtype=int32),
 array([0, 0, 3, ..., 3, 3, 0], dtype=int32),
 array([0, 0, 7, ..., 7, 7, 0], dtype=int32),
 array([1, 1, 3, ..., 3, 3, 1], dtype=int32),
 array([1, 1, 8, ..., 8, 8, 1], dtype=int32),
 array([1, 1, 7, ..., 7, 7, 1], dtype=int32),
 array([ 1,  1, 12, ..., 12, 12,  1], dtype=int32),
 array([1, 1, 7, ..., 7, 7, 1], dtype=int32),
 array([ 1,  1, 13, ..., 13, 13,  1], dtype=int32),
 array([ 0,  0, 13, ..., 13, 13,  0], dtype=int32),
 array([ 1,  1,  8, ...,  8, 16,  1], dtype=int32),
 array([ 1,  1,  0, ...,  0, 14,  1], dtype=int32),
 array([ 1,  1, 18, ..., 18, 18,  1], dtype=int32),
 array([ 1,  1, 15, ..., 15, 17,  1], dtype=int32)]

In [30]:
# definimos una lista donde vamos a guardar los Silhouette Scores para cada iteración
scores = []

El costo computacional del algoritmo que determina el Silhouette Score es muy alto, resultando inviable calcular el mismo sobre la totalidad de los puntos en nuestro dataset.
Dado que vamos a utilizar este Score para determinar el número óptimo de clusters para nuestro ejercicio, debemos además calcular este valor para cada una de las iteraciones.
Por tanto vamos a calcular el Silhouette sobre una muestra equivalente al 25% de los samples en nuestra base.

In [31]:
# cálculo del Silhouette Score para cada iteración
# fraccionamos esta corrida en varias partes para evitar que se interrumpa
start_time = timeit.default_timer()
for res in range (1, 5):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1469.0076419310008


In [32]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726]

In [33]:
start_time = timeit.default_timer()
for res in range (5, 10):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1969.2944157880002


In [34]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726,
 0.5630502930358205,
 0.5978683693759277,
 0.6250580606459699,
 0.6432568691490489,
 0.6600502843452842]

In [35]:
start_time = timeit.default_timer()
for res in range (10, 15):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1961.4701145640001


In [36]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726,
 0.5630502930358205,
 0.5978683693759277,
 0.6250580606459699,
 0.6432568691490489,
 0.6600502843452842,
 0.6759061821357895,
 0.6884294612294964,
 0.7028051851200853,
 0.715069498444841,
 0.7264878318867205]

In [37]:
start_time = timeit.default_timer()
for res in range (15, 20):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1969.889750671


In [38]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726,
 0.5630502930358205,
 0.5978683693759277,
 0.6250580606459699,
 0.6432568691490489,
 0.6600502843452842,
 0.6759061821357895,
 0.6884294612294964,
 0.7028051851200853,
 0.715069498444841,
 0.7264878318867205,
 0.7415511432966558,
 0.7432938673920847,
 0.7584287477966034,
 0.768563187520407,
 0.7729680171293019]

In [39]:
# cerrado el análisis hasta k=20 el SS muestra tendencia creciente
# continuamos iterandos el número de clusters en el rango 21-30
start_time = timeit.default_timer()
for num in range (21, 31):
  kmeans = KMeans(n_clusters=num, random_state=14)
  kmeans.fit(base2019_dummies)
  labels.append(kmeans.labels_)
elapsed = timeit.default_timer()-start_time
print(elapsed)

903.1988267200013


In [41]:
len(labels)

29

In [42]:
# evaluamos el SS para estos nuevos modelos
start_time = timeit.default_timer()
for res in range (20, 25):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1940.9509424619991


In [43]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726,
 0.5630502930358205,
 0.5978683693759277,
 0.6250580606459699,
 0.6432568691490489,
 0.6600502843452842,
 0.6759061821357895,
 0.6884294612294964,
 0.7028051851200853,
 0.715069498444841,
 0.7264878318867205,
 0.7415511432966558,
 0.7432938673920847,
 0.7584287477966034,
 0.768563187520407,
 0.7729680171293019,
 0.7814471546555716,
 0.792252780992064,
 0.7992741379647419,
 0.8065841218383616,
 0.8126162601459194]

In [44]:
# evaluamos el SS para estos nuevos modelos
start_time = timeit.default_timer()
for res in range (25, 30):
  scores.append(silhouette_score(base2019_dummies, labels[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

1785.0645131340007


In [45]:
scores

[0.33792724873206836,
 0.439997591339782,
 0.48861314408758105,
 0.5374049616104726,
 0.5630502930358205,
 0.5978683693759277,
 0.6250580606459699,
 0.6432568691490489,
 0.6600502843452842,
 0.6759061821357895,
 0.6884294612294964,
 0.7028051851200853,
 0.715069498444841,
 0.7264878318867205,
 0.7415511432966558,
 0.7432938673920847,
 0.7584287477966034,
 0.768563187520407,
 0.7729680171293019,
 0.7814471546555716,
 0.792252780992064,
 0.7992741379647419,
 0.8065841218383616,
 0.8126162601459194,
 0.8203013386463444,
 0.8233667021797498,
 0.8323457513637154,
 0.8369825970441148,
 0.8425210868839164]

Estoy empezando a sospechar que el resultado óptimo para el número de clusters es igual al número de categorías diferentes en la variable "concepto". <br>
Por las características de mi matriz de datos, si la diferencia en distancia geográfica no es significativa, no veo que el algoritmo tenga incentivos para agrupar categorías, siendo la solución óptima clasificar cada sample bajo la categoría reportada en concepto.<br>
Si en lugar de utilizar concepto utilizo un nivel de agrupación superior (categoría, subcategoría) para la clusterización, el resultado será similar, con un menor nivel de clusters. <br>
¿Podemos redefinir el problema y sacar un aprendizaje a partir de esto? Un ejemplo: a excepción de las dos categorías principales (Limpieza y recolección, Tránsito, que por su gran cantidad de casos es de esperar se distribuyan por toda la ciudad), ¿podemos ver un patrón de distribución de los contactos por categoría en alguna región geográfica en particular?

In [47]:
# por lo visto no estamos cerca de la cota superior
# pruebo un método más eficiente para ver la convergencia
kmeans100 = KMeans(n_clusters=100, random_state=14).fit(base2019_dummies)
kmeans100 = kmeans100.labels_ 

In [48]:
score100 = silhouette_score(base2019_dummies, kmeans100, sample_size=156525)
score100

0.942616033430552

In [49]:
# intentamos con valores más cerca del límite
kmeans200 = KMeans(n_clusters=200, random_state=14).fit(base2019_dummies)
kmeans200 = kmeans200.labels_ 

In [50]:
score200 = silhouette_score(base2019_dummies, kmeans200, sample_size=156525)
score200

0.5962918323210981

Una luz de esperanza!

In [51]:
# buscando el número óptimo
kmeans150 = KMeans(n_clusters=150, random_state=14).fit(base2019_dummies)
kmeans150 = kmeans150.labels_ 

In [52]:
score150 = silhouette_score(base2019_dummies, kmeans150, sample_size=156525)
score150

0.6981477481956232

NdA: aquí podría haber armado un lindo script usando if dentro de un loop para encontrar el valor óptimo, pero me temo que por el tiempo que demanda la corrida del SS colapse antes de entregar un resultado. <br>
Así que sigo buscando a mano.


In [53]:
# buscando el número óptimo
kmeans125 = KMeans(n_clusters=125, random_state=14).fit(base2019_dummies)
kmeans125 = kmeans125.labels_ 

In [54]:
score125 = silhouette_score(base2019_dummies, kmeans125, sample_size=156525)
score125

0.7719284023508081

In [55]:
# buscando el número óptimo
kmeans75 = KMeans(n_clusters=75, random_state=14).fit(base2019_dummies)
kmeans75 = kmeans75.labels_

In [56]:
score75 = silhouette_score(base2019_dummies, kmeans75, sample_size=156525)
score75

0.9290586071549265

In [57]:
# buscando el número óptimo
kmeans87 = KMeans(n_clusters=87, random_state=14).fit(base2019_dummies)
kmeans87 = kmeans87.labels_

In [58]:
score87 = silhouette_score(base2019_dummies, kmeans87, sample_size=156525)
score87

0.9364989647791528

In [59]:
# buscando el número óptimo
kmeans93 = KMeans(n_clusters=93, random_state=14).fit(base2019_dummies)
kmeans93 = kmeans93.labels_

In [60]:
score93 = silhouette_score(base2019_dummies, kmeans93, sample_size=156525)
score93

0.9396861812460464

In [61]:
# buscando el número óptimo
kmeans97 = KMeans(n_clusters=97, random_state=14).fit(base2019_dummies)
kmeans97 = kmeans97.labels_

In [62]:
score97 = silhouette_score(base2019_dummies, kmeans97, sample_size=156525)
score97

0.9412534944820398

In [63]:
labels_final = []

In [64]:
# a ver si con esto cerramos
start_time = timeit.default_timer()
for num in range (98, 105):
  kmeans = KMeans(n_clusters=num, random_state=14)
  kmeans.fit(base2019_dummies)
  labels_final.append(kmeans.labels_)
elapsed = timeit.default_timer()-start_time
print(elapsed)

1759.2950137280022


In [65]:
scores_final = []

In [69]:
start_time = timeit.default_timer()
for res in range (1, 8):
  scores_final.append(silhouette_score(base2019_dummies, labels_final[res-1], sample_size=156525))
elapsed = timeit.default_timer()-start_time
print(elapsed)

2488.9625945000007


In [70]:
scores_final

[0.7628545766109965,
 0.942014074717975,
 0.9425037158552472,
 0.9430208667536918,
 0.9432841002448835,
 0.7642576157923179,
 0.944328424221543]

OK, identificamos el número óptimo de clusters. Pero a fines del objetivo que nos propusimos, ¿tiene sentido hablar de 100 clusters? ¿Facilita esta división la jerarquización / atención de los incidentes reportados en los contactos? <br>
Pareciera ser más certero utilizar la agrupación en categorías (17 en total) o subcategorías (58) y ver en ese caso si llegamos a un número más reducido de clusters. (Pienso en el plot sobre el mapa, pareciera ser imposible la identificación de los 100 clusters si ploteamos en conjunto).

In [71]:
# a efectos de graficar los clusters obtenidos sobre el mapa, guardo en un DF la información relevante
base_plot = base2019_clean
base_plot['label'] = labels_final[6]
base_plot.head(5)

,lat,long,label
contacto,,,
00000001/19,-34.634060,-58.466561,1
00000002/19,-34.628162,-58.443378,1
00000003/19,-34.623324,-58.469810,12
00000005/19,-34.618464,-58.445304,1
00000006/19,-34.634200,-58.528355,1


In [73]:
base_plot.to_csv('/Users/fer/Downloads/DATASCIENCE/CLUSTER AI/GRUPO TP/SUACI/base_plot.csv')

In [77]:
#llegamos a 104 clusters y parece que podemos probar algunos más
#ojo con la aparición de minimos locales

In [ ]:
BBox = (base2019_dummies.lat.min(), base2019_dummies.lat.max(), base2019_dummies.long.min(), base2019_dummies.long.max())

In [ ]:
BBox

In [ ]:
#importamos el mapa
mapa = plt.imread('/Users/fer/Downloads/DATASCIENCE/CLUSTER AI/GRUPO TP/SUACI/mapa-CABA.png')

In [ ]:
#plot contactos
fig, ax = plt.subplots(figsize = (10,12))
ax.scatter(base2019_dummies.long, base2019_dummies.lat, zorder=1, alpha= 0.2, c='b', s=10)
ax.set_title('Distribucion contactos al SUACI 2019')
ax.set_xlim(BBox[2],BBox[3])
ax.set_ylim(BBox[0],BBox[1])
#ax.imshow(mapa, zorder=0, extent = BBox, aspect= 'equal')
plt.imshow(mapa, zorder=0, extent = BBox, aspect= 'equal')
plt.show()

In [ ]:
#  plot contactos, con distinción del cluster
plt.figure(figsize = (10,12))
sns.scatterplot(base2019_dummies.long, base2019_dummies.lat, hue = base2019_dummies.label, palette = 'Set2')
plt.show()